## Random Forest & XGBoost Models: Preporcessing Strategy ##

TRAIN SET:
- Handle missing values: impute Nan with -1 with missing_flag indicator; 
- Handle skeweness: not necessary;
- Encoding categorical variables (TargetEncoding), and scaling - not necessary;

TEST, VALIDATION SETS:
Apply the same sub-steps from above but with the parameters learned from the **training set**.


In [2]:
import pandas as pd

filename = "cleaned_properties.csv"
df = pd.read_csv(filename)
df.columns
df

,id,price,property_type,subproperty_type,region,province,locality,zip_code,latitude,longitude,...,fl_garden,garden_sqm,fl_swimming_pool,fl_floodzone,state_building,primary_energy_consumption_sqm,epc,heating_type,fl_double_glazing,cadastral_income
0,34221000,225000.0,APARTMENT,APARTMENT,Flanders,Antwerp,Antwerp,2050,51.217172,4.379982,...,0,0.0,0,0,MISSING,231.0,poor,GAS,1,922.0
1,2104000,449000.0,HOUSE,HOUSE,Flanders,East Flanders,Gent,9185,51.174944,3.845248,...,0,0.0,0,0,MISSING,221.0,poor,MISSING,1,406.0
2,34036000,335000.0,APARTMENT,APARTMENT,Brussels-Capital,Brussels,Brussels,1070,50.842043,4.334543,...,0,0.0,0,1,AS_NEW,NaN,MISSING,GAS,0,NaN
3,58496000,501000.0,HOUSE,HOUSE,Flanders,Antwerp,Turnhout,2275,51.238312,4.817192,...,0,0.0,0,1,MISSING,99.0,excellent,MISSING,0,NaN
4,48727000,982700.0,APARTMENT,DUPLEX,Wallonia,Walloon Brabant,Nivelles,1410,NaN,NaN,...,1,142.0,0,0,AS_NEW,19.0,excellent,GAS,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75503,30785000,210000.0,APARTMENT,APARTMENT,Wallonia,Hainaut,Tournai,7640,NaN,NaN,...,0,0.0,0,1,AS_NEW,NaN,MISSING,MISSING,1,NaN
75504,13524000,780000.0,APARTMENT,PENTHOUSE,Brussels-Capital,Brussels,Brussels,1200,50.840183,4.435570,...,0,0.0,0,0,AS_NEW,95.0,good,GAS,1,NaN
75505,43812000,798000.0,HOUSE,MIXED_USE_BUILDING,Brussels-Capital,Brussels,Brussels,1080,NaN,NaN,...,0,0.0,0,1,TO_RENOVATE,351.0,bad,GAS,0,NaN
75506,49707000,575000.0,HOUSE,VILLA,Flanders,West Flanders,Veurne,8670,NaN,NaN,...,1,NaN,0,1,AS_NEW,269.0,poor,GAS,1,795.0


In [3]:
#Define features (X) and target variable (y)

from sklearn.model_selection import train_test_split

X = df.drop(columns = ["price","id","zip_code","latitude","longitude"])
y = df["price"]
type(y)

pandas.core.series.Series

In [4]:
#Split the dataset into train, test, validation sets (e.g., 60/20/20 split);
from sklearn.model_selection import train_test_split

X_temp, X_test, y_temp, y_test = train_test_split(X,y, test_size=0.2, random_state = 86)

X_train, X_val, y_train, y_val = train_test_split(X_temp,y_temp, test_size = 0.25, random_state = 86)

In [5]:
print(type(X_train))
print(type(X_test))
print(type(X_val))
print(type(y_train))
print(type(y_test))
print(type(y_val))

<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>


## Random Forests & XGBoost Models 2-step Preprocessor ##

In [6]:
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler,FunctionTransformer, StandardScaler,OneHotEncoder
from sklearn.base import BaseEstimator, TransformerMixin
import category_encoders as ce
from category_encoders import TargetEncoder

In [ ]:
#NAN: replace NAN with -1 and add a missing_fl column here:
numeric_features = ["surface_land_sqm","total_area_sqm", "garden_sqm","terrace_sqm",
                    "cadastral_income", "surface_land_sqm", "construction_year",
                    "primary_energy_consumption_sqm","nbr_bedrooms","nbr_frontages",
                    "terrace_sqm", "total_area_sqm","garden_sqm"]

# No NAN to be handled, only encoding
categorical_onehot = ["property_type","region","province","heating_type","equipped_kitchen", "epc"]
categorical_target = ["subproperty_type","locality","state_building"]

# # No NAN to be handled and no encoding
binary_features = ["fl_floodzone", "fl_double_glazing", "fl_open_fire","fl_terrace", 
                        "fl_garden", "fl_swimming_pool", "fl_furnished"
                        ]


In [ ]:
# Pipeline for numeric columns (imputation)

numeric_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="constant", fill_value=-1, add_indicator=True))
])


# Pipeline